In [ ]:
import os
from openai import OpenAI
from pydantic import BaseModel, Field

In [ ]:
import logging
from typing import Optional
from datetime import datetime

# Defining data models for all the stages

In [ ]:
# Data model for extracting the event type
class ExtractEvent(BaseModel):
    
    description: str = Field(description="Basic description of event")
    is_calendar_event: bool = Field(
        description= "If the text describes a calendar event"
    )
    confidence_score: float = Field(description="Confidence score in the range of 0 to 1")

In [ ]:
#Data model for parsing specific details
class EventDetails(BaseModel):

    name: str = Field(description="Name of event")
    date: str = Field(description= "Date and time of the event. Use ISO 8601 to format this value.")

    duration_minutes: int = Field(description="Duration in minutes")
    participants: list[str] = Field(description= "Participant list")

In [ ]:
#Data model for confirmation message
class EventConfirm(BaseModel):
    confirm_message: str = Field(description= "Natural language message for confirmation")

    calendar_link: Optional[str] = Field(description= "Calendar link generated")

In [ ]:
#Setting up logging configuration
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model = "gpt-4o-mini"

# Defining the functions now

In [ ]:
def extract_event_info(user_input: str) -> ExtractEvent:
    """Determining if this is a calendar event by calling LLM"""
    logger.info("Analyzing the event extraction")
    logger.debug(f"Input text: {user_input} ")

    today = datetime.now()
    date_today = f"Today is {today.strftime('%A, %B, %d, %Y')}."

    completion = client.beta.chat.completions.parse(
        model=model,
        messages=[
            {
                "role": "system",
                "content": f"{date_today} Analyze if the extracted info points to a calendar event",
            },
        ],
        response_format=ExtractEvent,
    )
    result = completion.choices[0].message.parsed
    logger.info(
        f"Extraction done - Is calendar event: {result.is_calendar_event}, Confidence: {result.confidence_score:.2f} "
    )
    return result

In [ ]:
def event_info_parse(description: str) -> EventDetails:
    """Parsing the extracted event info with LLM"""
    logger.info("Parsing the event info")

    today = datetime.now()
    date_today = f"Today is {today.strftime('%A, %B, %d, %Y')}."

    completion = client.beta.chat.completions.parse(
        model=model,
        messages=[
            {
                "role": "system",
                "content": f"{date_today} Extract details from the event info. When dates refer to 'next Friday' or relative dates like this, use this date as current date for reference.",
            },
            {"role": "user", "content": description},
        ],
        response_format=EventDetails,
    )
    result = completion.choices[0].message.parsed
    logger.info(
        f"Parsed event info - Name:{result.name}, Date: {result.date}, Duration: {result.duration_minutes}min "
    )
    logger.debug(f"Participants: {', '.join(result.participants)} ")
    return result

In [ ]:
def get_confirmation(event_details: EventDetails) -> EventConfirm:
    """Getting a confirm message generated by calling LLN"""

    logger.info("Creating confirmation message")

    completion = client.beta.chat.completions.parse(
        model = model,
        messages=[
            {
                "role": "system",
                "content": "Generate a confirmation message for the event in natural human language. Sign: August",

            },
            {"role": "user", "content": str(event_details.model_dump()) },
        ],
        response_format=EventConfirm,
    )

    result = completion.choices[0].message.parsed
    logger.info("Confirmation message generated successfully")
    return result

# Now to chain the functions together

In [ ]:
def calendar_request_process(user_input: str) -> Optional[EventConfirm]:
    """This is the main function that implements the complete prompt chaining with gate check"""

    logger.info("Calendar request processing")
    logger.debug(f"Raw input: {user_input}")

    #Calling first LLM: This extracts the basic details
    initial_extract = extract_event_info{user_input}

    #Gate check: To check if it's a calendar event with enough confidence
    if(
        not initial_extract.is_calendar_event
        or initial_extract.confidence_score < 0.7
    ):
        logger.warning(
            f"Gate check failed - is_calendar_event: {initial_extract.is_calendar_event}, confidence: {initial_extract.confidence_score:.2f} "
        )
        return None
    
    logger.info("Gate check passed, now proceed with event process")

    #Second LLM: Get in depth details of event
    event_details = event_info_parse(initial_extract.description)

    #Third LLM: Get confirmation
    confirmation = get_confirmation(event_details)

    logger.info("Processing of calendar request completed successfully")
    return confirmation


# Now, Let's test the chain with valid input

In [ ]:
user_input = "Let's schedule a team metting for half an hour next Thursday at 1pm with Chris and Nina to discuss team budget."

result = calendar_request_process(user_input)
if result:
    print(f"Confirmation: {result.confirm_message}")
    if result.calendar_link:
        print(f"Calendar Link: {result.calendar_link}")
else:
    print("This isn't a calendar event request.")

# Now, let's test with an invalid input too

In [ ]:
user_input = "Can you set a reminder to send email to Nina?"

result = calendar_request_process(user_input)
if result:
    print(f"Confirmation: {result.confirm_message}")

    if result.calendar_link:
        print(f"Calendar Link: {result.calendar_link}")

else: 
    print("This doesn't look like a calendar event request")